In [211]:
import numpy as np
import pandas as pd
import os
import re
from io import StringIO

In [139]:
for folder in os.listdir('data'):
    with open(os.path.join('data_summaries', 'summary_{}.txt'.format(str(folder))), "w") as outfile:
        for textfile in os.listdir(os.path.join('data', folder)):
            contents = open( os.path.join('data', folder, textfile) ).read()
            outfile.write(contents)

In [144]:
data_summaries = pd.read_excel('data.xlsx')
file_list = os.listdir('data_summaries')

['summary_sub10.txt',
 'summary_sub12.txt',
 'summary_sub13.txt',
 'summary_Sub15.txt',
 'summary_Sub2.txt',
 'summary_Sub31.txt',
 'summary_Sub32.txt',
 'summary_sub5.txt',
 'summary_Sub6.txt',
 'summary_sub7.txt',
 'summary_sub8.txt']

In [228]:
type_changed = False
TYPE_ID = 0
TYPES_DF = pd.DataFrame()
entries = []
entry_num = 0
with open (os.path.join('data_summaries', file_list[0]), 'r') as text_file:
    string = text_file.readlines()
    for line in string:        
        for substring in ['MEASURMENT NR: ', 'TYPE SELECTED TO']:
            if substring in line:
                NUMBER = [int(s) for s in line.split() if s.isdigit()][0]
                if substring == 'TYPE SELECTED TO':
                    TYPE_ID_MEMORY = NUMBER
                    type_changed = True                    
                if (NUMBER == 1 and substring=='MEASURMENT NR: '):
                    if type_changed:
                        TYPE_ID = TYPE_ID_MEMORY
                    else:
                        TYPE_ID = 0
                    type_changed = False
                if substring == 'MEASURMENT NR: ':
                    TYPES_DF = TYPES_DF.append({'MEASUREMENT_NO':int(NUMBER), 'TYPE_ID':int(TYPE_ID)}, ignore_index=True)                        
        if 'DATA' in line:
            print(pd.read_csv( StringIO(str(entries)), sep='\t'))
            TYPES_DF['DATAFRAME'].iloc[entry_num]  = pd.read_csv( StringIO(str(entries)), sep='\t')
            entry_num +=1

Empty DataFrame
Columns: [[]]
Index: []


KeyError: 'DATAFRAME'

In [224]:
TYPES_DF

,DATAFRAME,MEASUREMENT_NO,TYPE_ID
0,Empty DataFrame Columns: [[]] Index: [],0.0,0.0
1,Empty DataFrame Columns: [[]] Index: [],1.0,0.0
2,Empty DataFrame Columns: [[]] Index: [],2.0,0.0
3,Empty DataFrame Columns: [[]] Index: [],3.0,0.0
4,Empty DataFrame Columns: [[]] Index: [],4.0,0.0
...,...,...,...
78,Empty DataFrame Columns: [[]] Index: [],5.0,9.0
79,Empty DataFrame Columns: [[]] Index: [],6.0,9.0
80,Empty DataFrame Columns: [[]] Index: [],7.0,9.0
81,Empty DataFrame Columns: [[]] Index: [],8.0,9.0


In [ ]:
MEASURMENT NR: 1


In [85]:
result = re.search('MEASURMENT(.*)', str(string))

In [86]:
result

<re.Match object; span=(78751, 1794529), match='MEASURMENT NR: 1\\n\', \'\\n\', \'2256.00\\t2269.>

SyntaxError: invalid syntax (<ipython-input-89-16e9534120c1>, line 1)